In [12]:
import yaml
from web3 import Web3
import json
import solcx
from solcx import install_solc

import json
import os
install_solc("0.8.9") # replace this with version of solidity to use

Version('0.8.9')

# Importing Keys

In [4]:
with open('../keys.yml', 'r') as file:
    keys = yaml.safe_load(file)

# Defining Public and Private Key

In [17]:
# Defining the Ethereum addresses of two accounts
account_1 = keys["ganache_test1"]["public_key"]
account_2 = keys["ganache_test2"]["public_key"]

# Defining the private key of the first account
private_key = keys["ganache_test1"]["private_key"]

# Connecting to a node

In [6]:
# Defining the URL of the node that will be interacting with the blockchain
infura_url = "HTTP://127.0.0.1:7545"

# Connecting the API to the node using the Web3 library
web3 = Web3(Web3.HTTPProvider(infura_url))

# Checking if the connection to the node was successful
if web3.isConnected():
    print("Successfully connected to the node at", infura_url)
else:
    print("Connection to node failed. Please check the URL and try again.")


Successfully connected to the node at HTTP://127.0.0.1:7545


# Creating Smart Contract

In [15]:
os.chdir('../smart_contract')

with open("../smart_contract/INR_Stable_Coin.sol", "r") as file:
    source_code = file.read()

# Compile the contract
compiled_contract = solcx.compile_source(source_code)

# Get the ABI and bytecode of the contract
contract_abi = compiled_contract["<stdin>:ChimereRupee"]["abi"]
contract_bytecode = compiled_contract["<stdin>:ChimereRupee"]["bin"]

os.chdir('../Notebooks')


In [16]:
with open("../smart_contract//ChimereRupee_abi.json", "w") as file:
    json.dump(contract_abi,file)
with open("../smart_contract//ChimereRupee_bytecode.pkl", "w") as file:
    file.write(str(contract_bytecode))

In [19]:
# Deploy the contract
contract = web3.eth.contract(
    abi=contract_abi,
    bytecode=contract_bytecode
)



# Getting the nonce (number of transactions sent) of the first account
nonce = web3.eth.getTransactionCount(account_1)


# Get the transaction hash of the deployment
transaction = contract.constructor().build_transaction(
                                                        {
                                                            "gasPrice": web3.eth.gas_price,
                                                            "from" : account_1,
                                                            "nonce" : nonce
                                                        }

                                                        )

In [20]:
#sign transaction
signed_transaction = web3.eth.account.sign_transaction(transaction, private_key = private_key)

#transction hash
tx_hash = web3.eth.send_raw_transaction(signed_transaction.rawTransaction)

# Wait for the transaction to be mined
tx_receipt = web3.eth.waitForTransactionReceipt(tx_hash)

# Get the address of the deployed contract
contract_address = tx_receipt["contractAddress"]

# Create a contract instance using the address of the deployed contract
greeter = web3.eth.contract(
    address=contract_address,
    abi=contract_abi
)


In [21]:
contract_address

'0x3A07a0bd2c2c1d6ECE4BBC4967D84F57a6153aAD'

# Executing Smart Contract

In [22]:

# Defining the ABI and address of the contract
abi = ""  # Replace with the ABI of your contract

with open("../smart_contract//ChimereRupee_abi.json","r") as file:
    abi = json.load(file)

contract_address = contract_address

address = web3.toChecksumAddress(contract_address)  # Replace with the address of your contract

# Creating a contract object using the Web3 library
contract = web3.eth.contract(address=address, abi=abi)

# Printing the current greeting in the contract
print(contract.functions.totalSupply().call())



0


In [25]:
# Printing the current greeting in the contract
print(contract.functions.balanceOf(account_2).call())
print(contract.functions.balanceOf(account_1).call())

0
0


In [26]:

# Getting the nonce (number of transactions sent) of the first account
nonce = web3.eth.getTransactionCount(account_1)


# Get the transaction hash of the deployment
transaction = contract.functions.mint(to=account_1,amount=100).build_transaction(
                                                                                {
                                                                                    "gasPrice": web3.eth.gas_price,
                                                                                    "from" : account_1,
                                                                                    "nonce" : nonce
                                                                                }

                                                                                )


#sign transaction
signed_transaction = web3.eth.account.sign_transaction(transaction, private_key = private_key)

#transction hash
tx_hash = web3.eth.send_raw_transaction(signed_transaction.rawTransaction)


# # Updating the greeting in the contract
# tx_hash = contract.functions.setGreeting('NEW GREETING!').transact()

# Waiting for the transaction to be mined
web3.eth.waitForTransactionReceipt(tx_hash)



AttributeDict({'transactionHash': HexBytes('0x1e56e9273f0e5cb5c82d42b64fbf7dd661c9eb70fb10bd9ccf2e0198d8164163'),
 'transactionIndex': 0,
 'blockHash': HexBytes('0xe08cff1441fb058b0c74788eebdffbed5b68cded407170c3bf12095f70cb4b43'),
 'blockNumber': 4,
 'from': '0x82dbbb95BEDB980E6bE1cCbc013B18c72af30149',
 'to': '0x3A07a0bd2c2c1d6ECE4BBC4967D84F57a6153aAD',
 'gasUsed': 67284,
 'cumulativeGasUsed': 67284,
 'contractAddress': None,
 'logs': [AttributeDict({'logIndex': 0,
   'transactionIndex': 0,
   'transactionHash': HexBytes('0x1e56e9273f0e5cb5c82d42b64fbf7dd661c9eb70fb10bd9ccf2e0198d8164163'),
   'blockHash': HexBytes('0xe08cff1441fb058b0c74788eebdffbed5b68cded407170c3bf12095f70cb4b43'),
   'blockNumber': 4,
   'address': '0x3A07a0bd2c2c1d6ECE4BBC4967D84F57a6153aAD',
   'data': '0x0000000000000000000000000000000000000000000000000000000000000064',
   'topics': [HexBytes('0xddf252ad1be2c89b69c2b068fc378daa952ba7f163c4a11628f55a4df523b3ef'),
    HexBytes('0x0000000000000000000000000000000

In [ ]:
# Printing the updated greeting in the contract
print('Updated Balance: {}'.format(contract.functions.greet().call()))


Updated greeting: Got the concept!


In [27]:
print(contract.functions.totalSupply().call())


100
